In [2]:

class BudgetAllocator:
    def __init__(self, budget):
        self.budget = budget
        self.essential_expenses = []  # (name, cost)
        self.non_essential_expenses = []  # (name, cost, importance)
        self.emergency_fund = 0.1 * budget  # 10% reserved for emergencies

    def add_expense(self, name, cost, essential=True, importance=0):
        if essential:
            self.essential_expenses.append((name, cost))
        else:
            self.non_essential_expenses.append((name, cost, importance))

    def optimize_budget(self):
        remaining_budget = self.budget - self.emergency_fund

        # Cover essential expenses first
        for name, cost in self.essential_expenses:
            if remaining_budget >= cost:
                remaining_budget -= cost
            else:
                return "Budget insufficient for essential expenses!"

        # Convert budget to integer for DP indexing
        remaining_budget = int(remaining_budget)

        # Use 0/1 Knapsack DP approach for non-essential expenses
        n = len(self.non_essential_expenses)
        dp = [[0] * (remaining_budget + 1) for _ in range(n + 1)]

        for i in range(1, n + 1):
            name, cost, importance = self.non_essential_expenses[i - 1]
            cost = int(cost)  # Ensure cost is an integer
            for b in range(remaining_budget, cost - 1, -1):  # Iterate backwards for space optimization
                dp[i][b] = max(dp[i - 1][b], dp[i - 1][b - cost] + importance)

        # Traceback to find selected non-essential expenses
        selected_expenses = []
        b = remaining_budget
        for i in range(n, 0, -1):
            name, cost, importance = self.non_essential_expenses[i - 1]
            cost = int(cost)
            if b >= cost and dp[i][b] == dp[i - 1][b - cost] + importance:
                selected_expenses.append((name, cost))
                b -= cost

        return {
            "essential_expenses": self.essential_expenses,
            "selected_non_essential_expenses": selected_expenses,
            "remaining_budget": b,
            "emergency_fund": self.emergency_fund
        }

    def handle_emergency(self, emergency_cost):
        if emergency_cost <= self.emergency_fund:
            self.emergency_fund -= emergency_cost
            return f"Emergency covered using fund. Remaining emergency fund: {self.emergency_fund}"
        else:
            deficit = emergency_cost - self.emergency_fund
            self.emergency_fund = 0
            return f"Emergency fund insufficient! Need to cut {deficit} from non-essentials."

# Example Usage
budget_manager = BudgetAllocator(1000)
budget_manager.add_expense("Rent", 400, essential=True)
budget_manager.add_expense("Food", 200, essential=True)
budget_manager.add_expense("Internet", 50, essential=True)
budget_manager.add_expense("Gym Membership", 50, essential=False, importance=5)
budget_manager.add_expense("Streaming Service", 20, essential=False, importance=3)
budget_manager.add_expense("Dining Out", 80, essential=False, importance=4)
budget_manager.add_expense("Gaming Subscription", 15, essential=False, importance=2)

optimized_budget = budget_manager.optimize_budget()
print(optimized_budget)

# Simulating an emergency
emergency_response = budget_manager.handle_emergency(80)
print(emergency_response)

{'essential_expenses': [('Rent', 400), ('Food', 200), ('Internet', 50)], 'selected_non_essential_expenses': [('Gaming Subscription', 15), ('Dining Out', 80), ('Streaming Service', 20), ('Gym Membership', 50)], 'remaining_budget': 85, 'emergency_fund': 100.0}
Emergency covered using fund. Remaining emergency fund: 20.0
